In [2]:
import pandas as pd
import numpy as np
import nltk
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from nltk.corpus import stopwords 
from sklearn.metrics import mean_absolute_error, mean_squared_error
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm

In [3]:
data= pd.read_csv("/train.csv")
data.head()

,index,beer/ABV,beer/beerId,beer/brewerId,beer/name,beer/style,review/appearance,review/aroma,review/overall,review/palate,review/taste,review/text,review/timeStruct,review/timeUnix,user/ageInSeconds,user/birthdayRaw,user/birthdayUnix,user/gender,user/profileName
0,40163,5.0,46634,14338,Chiostro,Herbed / Spiced Beer,4.0,4.0,4.0,4.0,4.0,Pours a clouded gold with a thin white head. N...,"{'min': 38, 'hour': 3, 'mday': 16, 'sec': 10, ...",1229398690,NaN,NaN,NaN,NaN,RblWthACoz
1,8135,11.0,3003,395,Bearded Pat's Barleywine,American Barleywine,4.0,3.5,3.5,3.5,3.0,12oz bottle into 8oz snifter.\t\tDeep ruby red...,"{'min': 38, 'hour': 23, 'mday': 8, 'sec': 58, ...",1218238738,NaN,NaN,NaN,NaN,BeerSox
2,10529,4.7,961,365,Naughty Nellie's Ale,American Pale Ale (APA),3.5,4.0,3.5,3.5,3.5,First enjoyed at the brewpub about 2 years ago...,"{'min': 7, 'hour': 18, 'mday': 26, 'sec': 2, '...",1101492422,NaN,NaN,NaN,Male,mschofield
3,44610,4.4,429,1,Pilsner Urquell,Czech Pilsener,3.0,3.0,2.5,3.0,3.0,First thing I noticed after pouring from green...,"{'min': 7, 'hour': 1, 'mday': 20, 'sec': 5, 'y...",1308532025,1.209827e+09,"Aug 10, 1976",208508400.0,Male,molegar76
4,37062,4.4,4904,1417,Black Sheep Ale (Special),English Pale Ale,4.0,3.0,3.0,3.5,2.5,A: pours an amber with a one finger head but o...,"{'min': 51, 'hour': 6, 'mday': 12, 'sec': 48, ...",1299912708,NaN,NaN,NaN,NaN,Brewbro000


descriptive statistics of whole data


In [4]:
data.describe()

,index,beer/ABV,beer/beerId,beer/brewerId,review/appearance,review/aroma,review/overall,review/palate,review/taste,review/timeUnix,user/ageInSeconds,user/birthdayUnix
count,37500.000000,37500.000000,37500.000000,37500.000000,37500.000000,37500.000000,37500.00000,37500.000000,37500.000000,3.750000e+04,7.856000e+03,7.856000e+03
mean,24951.887573,7.403725,21861.152027,3036.595120,3.900053,3.873240,3.88944,3.854867,3.922440,1.232794e+09,1.176705e+09,2.416303e+08
std,14434.009669,2.318145,18923.130832,5123.084675,0.588778,0.680865,0.70045,0.668068,0.716504,7.190955e+07,3.375514e+08,3.375514e+08
min,0.000000,0.100000,175.000000,1.000000,0.000000,1.000000,0.00000,1.000000,1.000000,9.262944e+08,7.034366e+08,-2.208960e+09
25%,12422.500000,5.400000,5441.000000,395.000000,3.500000,3.500000,3.50000,3.500000,3.500000,1.189194e+09,9.794810e+08,1.433628e+08
50%,24942.500000,6.900000,17538.000000,1199.000000,4.000000,4.000000,4.00000,4.000000,4.000000,1.248150e+09,1.100009e+09,3.183264e+08
75%,37416.750000,9.400000,34146.000000,1315.000000,4.500000,4.500000,4.50000,4.500000,4.500000,1.291330e+09,1.274973e+09,4.388544e+08
max,49999.000000,57.700000,77207.000000,27797.000000,5.000000,5.000000,5.00000,5.000000,5.000000,1.326267e+09,3.627295e+09,7.148988e+08


DROPPING UNNECESSARY COLUMN

In [5]:
df = data.drop(['review/timeStruct','review/timeUnix','user/ageInSeconds','user/birthdayRaw','user/birthdayUnix','user/gender','user/profileName','beer/brewerId','beer/beerId'],axis=1)
df

,index,beer/ABV,beer/name,beer/style,review/appearance,review/aroma,review/overall,review/palate,review/taste,review/text
0,40163,5.00,Chiostro,Herbed / Spiced Beer,4.0,4.0,4.0,4.0,4.0,Pours a clouded gold with a thin white head. N...
1,8135,11.00,Bearded Pat's Barleywine,American Barleywine,4.0,3.5,3.5,3.5,3.0,12oz bottle into 8oz snifter.\t\tDeep ruby red...
2,10529,4.70,Naughty Nellie's Ale,American Pale Ale (APA),3.5,4.0,3.5,3.5,3.5,First enjoyed at the brewpub about 2 years ago...
3,44610,4.40,Pilsner Urquell,Czech Pilsener,3.0,3.0,2.5,3.0,3.0,First thing I noticed after pouring from green...
4,37062,4.40,Black Sheep Ale (Special),English Pale Ale,4.0,3.0,3.0,3.5,2.5,A: pours an amber with a one finger head but o...
...,...,...,...,...,...,...,...,...,...,...
37495,35175,5.50,Blackberry Scottish-Style,Fruit / Vegetable Beer,4.0,3.5,3.5,3.5,3.5,12 oz brown longneck with no freshness dating....
37496,23666,8.50,Founders Dirty Bastard,Scotch Ale / Wee Heavy,4.5,4.0,3.5,4.5,4.5,A - A bright red with a maroon-amber hue; mini...
37497,47720,4.75,Stoudt's Fest,MÃ¤rzen / Oktoberfest,4.0,3.5,4.0,4.5,4.0,Sampled on tap at Redbones.\t\tThis marzen sty...
37498,33233,11.20,Founders KBS (Kentucky Breakfast Stout),American Double / Imperial Stout,4.0,4.0,4.0,5.0,5.0,Pours a black body with a brown head that very...


CORRELATION MATRIX

In [6]:
data_corr = data[['review/appearance','review/overall','review/palate', 'review/taste', 'beer/ABV']].corr()
data_corr

,review/appearance,review/overall,review/palate,review/taste,beer/ABV
review/appearance,1.000000,0.498733,0.555833,0.531676,0.288376
review/overall,0.498733,1.000000,0.697220,0.785220,0.197005
review/palate,0.555833,0.697220,1.000000,0.732092,0.350656
review/taste,0.531676,0.785220,0.732092,1.000000,0.354033
beer/ABV,0.288376,0.197005,0.350656,0.354033,1.000000


In [7]:
data_corr.reset_index(inplace=True)
data_corr = data_corr.melt(id_vars='index', value_vars=['review/appearance','review/overall','review/palate', 'review/taste', 'beer/ABV'],  value_name='corr', var_name='attribute')
data_corr

,index,attribute,corr
0,review/appearance,review/appearance,1.000000
1,review/overall,review/appearance,0.498733
2,review/palate,review/appearance,0.555833
3,review/taste,review/appearance,0.531676
4,beer/ABV,review/appearance,0.288376
5,review/appearance,review/overall,0.498733
6,review/overall,review/overall,1.000000
7,review/palate,review/overall,0.697220
8,review/taste,review/overall,0.785220
9,beer/ABV,review/overall,0.197005


SPLITTING THE DATASET INTO TRAIN AND TEST

In [8]:
X = df[['review/appearance' ,'review/aroma' ,'review/palate' ,'review/taste','beer/ABV']]
y = df[['review/overall']]
train_X, test_X, train_y, test_y = train_test_split(X, y, random_state=42)

APPLYING MULTIPLE REGRESSION

In [9]:
linear_regression = LinearRegression()
linear_regression.fit(train_X,train_y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [10]:
pred= linear_regression.predict(test_X)

CALCULATING MSE,MAE,RMSE

In [11]:
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(test_y, pred))
print('Mean Squared Error:', metrics.mean_squared_error(test_y, pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(test_y, pred)))

Mean Absolute Error: 0.3033285000237453
Mean Squared Error: 0.16111008613418487
Root Mean Squared Error: 0.40138520916220227


EFFECTS OF EACH INDEPENDENT VARIABLE ON DEPPENDENT VARIABLE 

In [12]:
coef=linear_regression.coef_
coef

array([[ 0.06607413,  0.09231651,  0.26558999,  0.54354021, -0.04218782]])

AS WE CAN SEE ABV(i.e VOLUME) OF ALCOHOL HAVE NEGATIVE EFFECT ON THE RATING WHEREAS REVIEW/TASTE HAS MAXIMUM IMPACT ON RATINGS THEN REVIEW/PALATE , BUT OTHER 2 VARIABLE ARE NOT AFFECTING MUCH.


In [13]:
import nltk 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [16]:
#df['review/text']=df['review/text'].str.lower()
#import string
#df['review/text'] = df['review/text'].str.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation)))
#df['review/text']

2. EFFECT OF REVIEW/TEXT ON OVERALL_RATINGS
  (TEXT PREPROCESSING)

In [17]:
stop_words=stopwords.words('english')
wordnet_lemmatizer = WordNetLemmatizer()
porter=PorterStemmer()

def preprocess(text):
    
    text=str(text)
    clean_text1=text.lower()
    clean_text1=re.sub('[^a-zA-z]',' ',clean_text1)
    clean_text1=word_tokenize(clean_text1)
    clean_text1 = [i for i in clean_text1 if not i in stop_words]
    #clean_text1 = [snowball_stemmer.stem(word) for word in text]
    clean_text1=[wordnet_lemmatizer.lemmatize(word,pos="v") for word in clean_text1]
    clean_text1=" ".join(clean_text1)
    return(clean_text1)

df['text1'] = df['review/text'].apply(preprocess)
df['text1']


0        pour cloud gold thin white head nose quite flo...
1        oz bottle oz snifter deep ruby red hue one fin...
2        first enjoy brewpub years ago finally manage g...
3        first thing notice pour green bottle glass sku...
4        pour amber one finger head strong pour head re...
                               ...                        
37495    oz brown longneck freshness date pour sa pint ...
37496    bright red maroon amber hue minimal light tan ...
37497    sample tap redbones marzen style beer pour cle...
37498    pour black body brown head quickly fizzle away...
37499    nice sweet malty beer nothing complex smooth r...
Name: text1, Length: 37500, dtype: object

In [21]:
Train_X,Test_X,Train_Y,Test_Y=train_test_split(df['text1'], df['review/overall'], test_size=0.3)
Train_X

1600     fairly close black fluffy dark tan head stout ...
12357    clear amber orange well define medium large si...
8767     b r numerous occasion bomber pour standard pin...
35757    base note earlier version date bottle pakc car...
18692    pick michigan holiday look little viscous pour...
                               ...                        
33450    ounce bottle foodery pour duvel tulip bottle i...
27364    thank go bwyker crack bottle open jet black mo...
4611     pale lager beer like many others whitout parti...
32216    pour felt somewhat cheat since say stout look ...
18446    pour oily pure black rocky mocha head great re...
Name: text1, Length: 26250, dtype: object

TF-IDF WEIGHTING

In [22]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(df['text1'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

MULTINOMIAL NAIVE-BAYES

In [24]:
clf = MultinomialNB(alpha=0.1)
last_score = 0
for alpha in np.arange(0,1,.1):
    nb_classifier = MultinomialNB(alpha=alpha)
    Train_Y = Train_Y.astype('int')
    Test_Y =Test_Y.astype('int')
    nb_classifier.fit(Train_X_Tfidf, Train_Y)
    pred = nb_classifier.predict(Test_X_Tfidf)
    score = accuracy_score(pred,Test_Y)*100
    if score > last_score:
        clf = nb_classifier
    print("Alpha: {:.2f} Score: {:.5f}".format(alpha, score))

/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:507: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


Alpha: 0.00 Score: 63.91111
Alpha: 0.10 Score: 64.01778
Alpha: 0.20 Score: 64.05333
Alpha: 0.30 Score: 63.97333
Alpha: 0.40 Score: 63.90222
Alpha: 0.50 Score: 63.76000
Alpha: 0.60 Score: 63.62667
Alpha: 0.70 Score: 63.53778
Alpha: 0.80 Score: 63.40444
Alpha: 0.90 Score: 63.25333


PASSIVE AGGRESSIVE CLASSIFIER

In [25]:
linear_clf = PassiveAggressiveClassifier(max_iter=50)
linear_clf.fit(Train_X_Tfidf, Train_Y)
pred = linear_clf.predict(Test_X_Tfidf)
score = accuracy_score(Test_Y, pred)*100
print("accuracy:   %0.3f" % score)

accuracy:   59.102


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


SVM

In [ ]:
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)